In [ ]:
import sys
import os
import re
import copy
import numpy as np
import scipy as sp
from scipy.interpolate import interp1d
import math 
import random
import itertools
from typing import List, Optional 
from os import listdir
from os.path import isfile, join
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate, optimize
import multiprocessing
import pickle
from tqdm.notebook import tqdm

# load the various FastSim libraries
import Detector as Detector
import Particle as Particle
import Vertex as Vertex
import lorentz_transformation as lt
import llp_gun as llp_gun
## depending on which fvs are used load a different specialized llp gun (llp_gun_new) (built on top of llp_gun)
import llp_gun_new as lg
## NOTE: the various hadronic functions in llp_gun_new must be updated with the correct path of the LLP hadronic decay 
## 4-vector files depending on the type of analysis 

## These hadronic functions are almost the same but were separated into different functions for ease of use with the 
## different LLP analyses - but should be very easy to update them for any new analysis that need be done


### IMPORTANT ###
# All the hadronic decay 4-vector files are in the rest frame of the parent LLP (see files) and the code is set-up to boost them 
# to lab frame and deal with the same

# To use hadronic decay 4-vector files not in the rest frame of the parent LLP, one needs to comment/uncomment certain lines 
# in the llp_gun.py file 

## We give a demo of some basic functionalities i.e. creating an LLP particle or vertex manually, loading an LLP vertex from file and generating the event display. We also give a first look at the reconstruction/trigger capabilities of FastSim

In [ ]:
## LLP PARTICLE MANUALLY

# Let's start with some code taken from the get_weight function itself

## First, let us setup the detector by choosing a param_card
param_card = "param_card_CDR.txt"
# change this to the path to param_card on your local computer
path_to_param_card = "/Users/jai/Desktop/MATHUSLA_TOTAL/MATHUSLA_1/param_cards_smsrhn/" + param_card

# setup detector
detector_benchmark = Detector.Detector(path_to_param_card)
# lets see what the detector configuration is (might be messy, feel free to comment out)
print(detector_benchmark.config)

# clear the detector of past events, if any
detector_benchmark.clear_detector()

# choose a position for the LLP to start at - we just choose the interaction point (IP) and the sim
# automatically uses kinematics to figure out trajectory
position = (0,0,0)

# create an LLP particle, Particle.Particle(position, 4-vector, PID)
# play around with various 4-vectors and positions!
four_p = (100, 0, 100, 60) # (E, px, py, pz)
pid = 13 # let's say it is a muon (try changing the pid to an inivisble particle, say PID = 12, electron-neutrino)

llp = Particle.Particle(position, four_p, pid)

# feed this particle to the detector as a new (particle) event - other options include new_vertex_event which 
# feeds a vertex (i.e. an LLP with its decay products) to the detector
detector_benchmark.new_particle_event(llp)

# show/save event display (if needed) using the following (MORE information in the next few blocks)
# setting argument show = True displays and saves
# show = False saves but does not display
filename = "/Users/jai/Desktop/hello_MATHUSLA"
detector_benchmark.detector_display(filename, show=True)

# now lets get some information about this particle after it passed through detector

# get the current particle in the detector
par = detector_benchmark.return_current_particle() 
# just print the object to get info
print(par)

In [ ]:
# LLP VERTEX MANUALLY

# same as before
param_card = "param_card_CDR.txt"
path_to_param_card = "/Users/jai/Desktop/MATHUSLA_TOTAL/MATHUSLA_1/param_cards_smsrhn/" + param_card

detector_benchmark = Detector.Detector(path_to_param_card)
detector_benchmark.clear_detector()

position = (0,0,0)
four_p = (100, 0, 100, 60) # random 4-vector (might not equal sum of daughter 4-vectors, but we use it for demo anyway)
pid = 1023

# create daughter particles for the vertex 
daughter1 = Particle.Particle(position, (6, 0.2, -2.1, 3.7), 13)
daughter2 = Particle.Particle(position, (0.7, 0, 0.3, 0.4), 211)
daughter3 = Particle.Particle(position, (2, 0.6, 1, -0.2), 13)

decay_products = [daughter1, daughter2, daughter3]

# create the vertex
vertex = Vertex.Vertex(position, four_p, pid, decay_products)
detector_benchmark.new_vertex_event(vertex)

filename = "/Users/jai/Desktop/hello_MATHUSLA"
detector_benchmark.detector_display(filename, show=True)

# now lets get some information about this vertex after it passed through detector

# get the current vertex in the detector
ver = detector_benchmark.return_current_vertex() 
# just print the object to get info
print(ver)

In [ ]:
# LLP VERTEX FROM FILE

# same as before
param_card = "param_card_CDR.txt"
path_to_param_card = "/Users/jai/Desktop/MATHUSLA_TOTAL/MATHUSLA_1/param_cards_smsrhn/" + param_card

detector_benchmark = Detector.Detector(path_to_param_card)

##################
# change this to the path to bb_15.txt on your local machine
path_to_file = "/Users/jai/Desktop/H_hadronic_decays/bb_15.txt"
position = (0,0,0)
num = 2 # number of vertices you want to be read in from the given file

# get vertex (in LLP rest frame)
vertices = llp_gun.create_llp_from_file(path_to_file, position, num) # returns a list

# let's choose one of these, say the first one
vertex = vertices[0]

# feed it in 
detector_benchmark.clear_detector()
detector_benchmark.new_vertex_event(vertex)

filename = "/Users/jai/Desktop/hello_MATHUSLA"
print("Non-boosted LLP vertex\n")
detector_benchmark.detector_display(filename, show=True)
# you should notice that while their are a lot of "Particle trajectory out of bound." messages printed, 
# only 3 daughter particles are visible - this is because the others go in the direction opposite/out of the display
# frame

# what if we want to boost the vertex to say some detector frame
# we can aim the LLP at some specific starting target and boost LLP momentum to new momentum if specified as follows
target = (0,120,70)
p_norm = 100 # momentum LLP vertex (i.e. decay products) should be boosted to, we use the masses buit into 
#              vertex.decay_product to get the boost for each daughter
boosted_vertex = llp_gun.align_trajectory(vertex, target, p_norm)

# feed in vertex
detector_benchmark.clear_detector()
detector_benchmark.new_vertex_event(boosted_vertex)
# as expected we should end up with all the daughters (there should be more than the 3 visible before!) 
# boosted forward as expected

filename = "/Users/jai/Desktop/hello_MATHUSLA"
print("Boosted LLP vertex\n")
detector_benchmark.detector_display(filename, show=True)

## if we look at the top right figure, it is clear that there are tracks passing through ALL the ceiling sensor planes,
## however the top middle figure does not show the same picture, in fact, it might seem as if the tracks do not pass
## through ALL the ceiling sensors -- THIS IS JUST AN ARTIFACT OF THE MATPLOTLIB LIBRARY NOT BEING ABLE TO PLOT 2D
## PROJECTIONS OF 3D OBJECTS "NICELY", AND AS SUCH OVERLAYS/INTERSECTIONS ARE NOT EXACT ENOUGH - in this figure,
## the detector planes always overlay the tracks

## there is a work-around in the code and can be accessed by setting the argument zorder in the detector_display
## function to zorder = 1000 (so zorder only takes TWO values, 10 (default) or 1000)

detector_benchmark.detector_display(filename, show=True, zorder=1000)

## as you might see, now it's the tracks that overlay the detector planes

## both these views are provided as a sanity check for the user.


In [ ]:
## CHECKING IF RECONSTRUCTION/TRIGGER CRITERIA ARE PASSED
## by now we know how to create an LLP vertex, let's learn how to check if it is actually "detected" or not

param_card = "param_card_CDR.txt"
path_to_param_card = "/Users/jai/Desktop/MATHUSLA_TOTAL/MATHUSLA_1/param_cards_smsrhn/" + param_card

detector_benchmark = Detector.Detector(path_to_param_card)
detector_benchmark.clear_detector()

position = (0,0,0)
pid = 1023
four_p = (100, 0, 100, 60)

daughter1 = Particle.Particle(position, (6, 0.2, -2.1, 3.7), 13)
daughter2 = Particle.Particle(position, (0.7, 0, 0.3, 0.4), 211)
daughter3 = Particle.Particle(position, (2, 0.6, 1, -0.2), 13)

decay_products = [daughter1, daughter2, daughter3]


vertex = Vertex.Vertex(position, four_p, pid, decay_products)
# let's say the LLP vertex decayed at the point (0,120,70) with p_norm = 100
target = (0,120,70)
p_norm = 100 
vertex = llp_gun.align_trajectory(vertex, target, p_norm)

detector_benchmark.new_vertex_event(vertex)

filename = "/Users/jai/Desktop/hello_MATHUSLA"
detector_benchmark.detector_display(filename, show=True)

# upto this point everything was same as before

# how to check if LLP vertex passed DVmedium2 (DV2, see paper) recon criteria
if detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium2"): # gives 0 or 1 
    print("passed DV2")
else:
    print("did not pass DV2")

# how to check if LLP vertex passed DVmedium2 (DV2, see paper) recon criteria
if detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium3"): # gives 0 or 1 
    print("passed DV3")
else:
    print("did not pass DV3")

# how to check if LLP vertex passed trigger (nearest_neighbour, see paper) criteria
# NOTE: we use detector_benchmark.event_pass_trigger(trigger_criteria) without any argument as there is only one criteria
# and the simulation automatically uses that as default
if detector_benchmark.event_pass_trigger():
    print("passed trigger")
else:
    print("did not pass trigger")
    
# Finally we can take a product of recon and trigger to find whether LLP vertex passed both
if detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium2") * detector_benchmark.event_pass_trigger():
    print("passed both recon + trigger")
else:
    print("did not pass recon or trigger or maybe both")

## Now we give an explicit demo to get the geometric efficiency using the SM + S case

### First let's set-up some helper functions, which implement a lot of the functionality mentioned in Section III of the paper

In [ ]:
## read_vectors to be used with 4-vector in the format (E, px, py, pz)
## read_vectors_SMS to be used with 4-vectors in the format (weight, E, px, py, pz), where weight 
## is the number of real 4-vectors represented by given 4-vector (NOTE: SMS in the title, does
## not mean it should be only used with the SM+S case)

## argument "n" is the number of 4-vectors one needs, n = -1 reads in the entire file

def read_vectors(path: str, n=-1) -> List:
    # create llps using Higgs decay 4-vectors.
    vectors = []
    file = open(path, "r")
    for line in tqdm.tqdm_notebook(file.readlines()[1:]):
        # split string and convert to four-vector of ints
        # skip over first index of elements of clean as they are serial number
        m = list(map(float, line.strip().split(",")[1:]))
        # data is for beam axis along z, our beam axis is y so update coordinates
        m[1], m[2], m[3] = -m[1], m[3], m[2]
        vectors.append(np.array(m))
    if n == -1:
        return vectors
    else:
        return random.sample(vectors, n)

def read_vectors_SMS(path: str, n=-1) -> List:
    # create llps using Higgs decay 4-vectors.
    vectors = []
    file = open(path, "r")
    for line in file.readlines():
        # split string and convert to four-vector of ints
        m = list(map(float, line.strip().split(",")))
        w = m[0] # get the weight
        m = m[1:] # get the fv
        # data is for beam axis along z, our beam axis is y so update coordinates
        m[1], m[2], m[3] = -m[1], m[3], m[2]
        vectors.append(np.array([w] + m)) # fvs are of form (w, fv) - same convention throughout code
    if n == -1:
        return vectors
    else:
        return random.sample(vectors, n) # randomly sample

In [ ]:
## These functions take in 3-vectors and return there phi and theta angles

## get_theta is used to check if an LLP (4-vector) passes through the detector or not

## deal_with_phi takes advantage of rotational symmetry about the beam axis to randomly 
## choose a phi

# beam axis is y-axis
# theta is angle to y-axis i.e. arccos(y/r)
# phi is angle to x in x-z plane i.e arctan(x/z) 

def get_phi(u) -> float:# u is a three-vector 
    phi = np.arctan2(u[0], u[2]) # (x, z)
    return phi 

def get_theta(u) -> float:
    return np.arccos(u[1]/np.linalg.norm(u)) # (y/r)

def deal_with_phi(four_p: List[float], phi_min: float, phi_max: float) -> List[float]:
    
    curr_phi = get_phi(four_p[1:])
    new_phi = random.uniform(phi_min, phi_max)
    p_rot = lt.rotation(four_p[1:], np.array([0,1,0]), new_phi - curr_phi)
    new_4p = np.array([four_p[0], p_rot[0], p_rot[1], p_rot[2]])

    return new_4p

In [ ]:
## MAIN FUNCTION

## this function takes in an LLP 4-vector and if it passes through the detector, returns
## a weight i.e. probability of decay in the detector (according to formula mentioned in paper),
## decay position (w.r.t. the exponential CDF of the probability) and its boost. Returns None is LLP 4-vector
## does not pass through detector.

## NOTE: a lot of the components of this function can be used by themselves as separate functions
## if need be depending on the context

# ctau is lifetime, might be input by hand or given by reading in a file
# detector_benchmark is a Detector object, which we show how to initialize later
def get_weight(four_p: List[float], mass: float, ctau: float, detector_benchmark: Detector.Detector) -> Optional[tuple]:
    
    ## first we check if 4-vector passes through the detector and if it does, obtain its boundary hit coordinates i.e.
    ## entry (L1) and exit points (L2)
    
    # clear the detector of past events, if any
    detector_benchmark.clear_detector()
    # choose a position for the LLP to start at - we just choose the interaction point (IP) and the sim
    # automatically uses kinematics to figure out trajectory
    position = (0,0,0)
    # create an LLP particle, Particle.Particle(position, 4-vector, PID) - choose a random PID
    llp = Particle.Particle(position, four_p, 13)
    # feed this particle to the detector as a new (particle) event - other options include new_vertex_event which 
    # feeds a vertex (i.e. an LLP with its decay products) to the detector
    detector_benchmark.new_particle_event(llp)

    # set up a list to collect boundary hit point coordinates
    L1L2 = []
    
    # this list is a short-hand way of coordinates of the detector's boundary planes
    decay_vol_boundaries=['x-','x+','y-','y+','z-','z+']
    
    # if LLP passes through the detector
    if (detector_benchmark.track_is_in_decay_volume()):
        
        # get all the sensor layer hits in within the decay volume
        current_decay_volume_intersections = detector_benchmark.return_current_particle().decay_volume_hits

        # collect all the boundary hit objects
        decay_vol_boundaries_hits=[]
        
        # going through each of the boundary planes, check if there is a hit and collect it
        for i in decay_vol_boundaries:
            if current_decay_volume_intersections[i][0]:
                decay_vol_boundaries_hits.append(current_decay_volume_intersections[i][1])

        # another sanity check, there should be 2 hits, entry and exit
        if (len(decay_vol_boundaries_hits) == 2):
            # now convert list of hit coordinates into distance from the origin/IP
            L1L2 = [math.sqrt(item[0]**2 + item[1]**2 + item[2]**2) for item in decay_vol_boundaries_hits]
            L1L2.sort()
            # L1 is distance of entry point to IP
            # L2 is distance of exit point to IP
            L1,L2 = L1L2
            # sort the decay_vol_boundaries_hits so that the first (second) entry corresponds to coordinates of L1 (L2)
            decay_vol_boundaries_hits.sort(key=lambda x:math.sqrt(x[0]**2 + x[1]**2 + x[2]**2))
            
            # get the boost
            b = np.linalg.norm(four_p[1:])/mass 
            
            # now using random variable PDF inversion (using uniform distribution) get an exponential distribution for 
            # decay probability between L1 and L2
            unif = random.uniform(1-np.exp(-L1/(b*ctau)), 1-np.exp(-L2/(b*ctau)))
            exp = -b*ctau*np.log(1-unif) # use pdf inversion to get exponential distribution
            # explicitly get the decay position by multiply unit vector along L1 with the chosen norm from the exp distribution
            decay_position = np.array(decay_vol_boundaries_hits[0])/L1 * exp
            
            # get the weight i.e. probability of decay within the detector volume
            weight = np.exp(-L1/(b*ctau)) - np.exp(-L2/(b*ctau))

            return weight, decay_position, b
        
        # or return None if LLP does not pass through the detector
        else: 
            return None

## Putting the above functions to use (might take a while to fimish running!)

NOTE: Once again, a lot of the parts of the block below can be used as independent functions by themselves, we just give an example of how we used the entire code to get data for the SM+S case.

In [ ]:
# choose a mass for the LLP (in GeV)
m = 1.0

# choose a param card - we choose the default CDR geometry
card = "param_card_CDR.txt"

# setup a list of possible coupling values to be probed in the sim
thetas = 10**(np.linspace(-7,-1,num=121))
    
## now we setup the pipeline which takes into account all possible decays and their
## branching ratios
      
# use decay product masses as threholds for various types of decays
m_e = 0.511e-3 
m_mu = 0.1056 
m_pi = 0.13957039 
m_pi0 = 0.1349768 

# make a dictionary of the mass ranges and possible decays 
# ranges are based on Br figure from the source paper + make sure that the decay file for the mass exists
decays = {(2*m_e,2*m_mu):{"e":"leptonic2body"}, (2*m_mu,2*m_pi0):{"mu":"leptonic2body"}, 
          (2*m_pi0,0.7):{"mu":"leptonic2body","pi":"leptonic2body"}, 
          (0.7,0.99):{"mu":"leptonic2body", "hadrons0.7to0.98gev":"hadronic"}, 
          (1,2):{"mu":"leptonic2body", "hadrons1to2gev":"hadronic"},(2, 3.6):{"g":"hadronic", "s":"hadronic", "mu":"leptonic2body"}, (3.6, 3.8):{"g":"hadronic", "s":"hadronic", "tau":"hadronic", "mu":"leptonic2body"}, (3.8, 5):{"g":"hadronic", "s":"hadronic", "c":"hadronic", "tau":"hadronic", "mu":"leptonic2body"}}

# read in digitized data i.e. branching ratios and lifetime (in metres) of the LLP
br = {}
# change path to local machine
path = "/Users/jai/Desktop/MATHUSLA_TOTAL/SM+S/"
fnames = [f for f in os.listdir(path) if isfile(join(path, f))]
for fname in fnames:
    if fname != ".DS_Store": # this line is specifically for MacOS 
        br[fname] = np.loadtxt(path + fname, delimiter=',')
        br[fname] = interpolate.interp1d(br[fname][:,0], br[fname][:,1],fill_value="extrapolate")

# now we go through the all possible decays and choose the possible decays for the given mass, m
possible_decays = None
for mass_range in decays:
    if mass_range[0] <= m < mass_range[1]: # if condition should always be satisfied for atleast one mass range
        possible_decays = decays[mass_range]

if len(possible_decays) == 1: # i.e. e or mu 100% Br
    possible_brs = {list(possible_decays.keys())[0]:1}
elif 2*m_pi0 < m < 2*m_pi:
    brmu = float(br["Smu.csv"](m))
    possible_brs = {"mu":brmu, "pi":1-brmu}
elif "hadrons0.7to0.98gev" in possible_decays:
    brmu = float(br["Smu.csv"](m))
    possible_brs = {"mu": brmu, "hadrons0.7to0.98gev": 1-brmu}
elif "hadrons1to2gev" in possible_decays:
    brmu = float(br["Smu.csv"](m))
    possible_brs = {"mu": brmu, "hadrons1to2gev": 1-brmu}
else:
    possible_brs = {}
    for name in br:
        if name[1:-4] in possible_decays: # if condition should always be satisfied
            possible_brs[name[1:-4]] = float(br[name](m))
    # renormalize the Br to 1 at each mass point (as we ignore certain very low-chance decays)
    factor = np.sum(list(possible_brs.values()))
    for key in possible_brs:
        possible_brs[key] = possible_brs[key]/factor 

## now we have a dictionary, possible_brs whose keys are the possible decay name and values are
## corresponding branching ratios

################## now preliminaries are done ##################

# read in all the LLP 4-vectors from the SM+S LLP file (need all as the sum of the 4-vector
# weights adds up to xsec * lumi i.e. the actual number of SM+S generated LLPs generated at for e.g. CERN)

# change path to local machine i.e. where you store "SMS_LLPweight4vectorBmesonlist_mS_1.0.csv"
fv_path = "/Users/jai/Desktop/All_SMS_LLPweight4vectors_from_B"
fv_name = "SMS_LLPweight4vectorBmesonlist_mS_" + str(m) + ".csv"

# use the read_vectors_SMS file to read in, as format of 4-vectors is (weight, E, px, py, pz)
vectors = read_vectors_SMS(os.path.join(fv_path,fv_name), -1) # 4-vectors from B parents
# NOTE: for other types of LLP models such as RHN_Ui might have to read in 4-vectors from other sources 
# too like D mesons, W-,Z-bosons etc.

# finally, note length of read-in 4-vector list as we use this to iterate through this list later
num_iter = len(vectors)

# finally run the simulation - setup the detector by giving it the path to the chosen param_card
# change path to local machine
detector_benchmark = Detector.Detector("/Users/jai/Desktop/MATHUSLA_TOTAL/MATHUSLA_1/param_cards_smsrhn/" + card)

# find max min angles OF THE DECAY VOLUME (note its not the detector volume!)
x = np.array([detector_benchmark.config.decay_x_min, detector_benchmark.config.decay_x_max])
y = np.array([detector_benchmark.config.decay_y_min, detector_benchmark.config.decay_y_max])
z = np.array([detector_benchmark.config.decay_z_min, detector_benchmark.config.decay_z_max])

corners = np.array(np.meshgrid(x, y, z)).T.reshape(-1,3)
points = corners.copy()

detector_theta = []
detector_phi = []

# for efficiency, randomly generate 100 points lying in/on the detector and take min,max angles from these
# -- ends up giving the (almost) actual min, max (using Central Limit Theorem)
for j in itertools.product(corners, corners):
    for k in range(100):
        u = random.uniform(0,1)
        new = u * j[0] + (1-u) * j[1]
        detector_theta.append(get_theta(new))
        detector_phi.append(get_phi(new))

theta_min, theta_max = min(detector_theta), max(detector_theta)
phi_min, phi_max = min(detector_phi), max(detector_phi)

# now we have angle ranges for MATHUSLA and can compare LLP 4-vectors (with starting point being the IP)
# with the same to check if they pass through the detector decay volume

# iterate through each possible coupling we want to probe
for index in tqdm(range(len(thetas))): 
    print("starting new theta value")
    theta = thetas[index]

    # dictionary to collect geometric efficiency under different criteria
    eff = {"perfect":0,"DVmedium2":0,'DVmedium3':0,"DVmedium2loose1":0,"DVtight2":0,
          'DVsupertight2':0,'DVtight1medium1':0,'DVtight1loose1':0,'DVtight3':0,
          'DVtight2medium1':0,'DVtight1medium2':0,'DVtight1loose2':0}

    # official c = 299,792,458 m/s
    c = 299792458
    
    # read in ctau corresponding to our chosen mass, m, from a file
    # NOTE: there are two files we read in from depending on the mass, thus the if else block
    if m in [0.03, 0.0774597, 0.2, 0.212]:
        ctau = c * br["Slife_old.csv"](m) * (theta**(-2)) * 10**(-12)
    else:
        # ctau = c(m/s) * tau(s) * coupling factor
        ctau = c * br["Slife_new.csv"](m) * (theta**(-2))

    ################## actual simulation ################ 
    # iterate through the LLP 4-vectors
    for k in tqdm(range(num_iter)): 
        # first entry is weight
        four_pw = vectors[k] # fv[0] is weight 
        # reweigh the weight depending on the coupling (MODEL SPECIFIC!)
        # reweigh by sin_theta^2 ~ theta^2 in our case, as the Br depends on theta
        w = four_pw[0] * theta**2 
        # keep track of actual 4-vector
        four_p = four_pw[1:]
        # get the LLP theta and phi angles 
        llp_theta = get_theta(four_p[1:])
        llp_phi = get_phi(four_p[1:])
        # check if LLP passes theta (equivalently eta) range of MATHUSLA i.e. it has chance of passing through
        ## (NOTE: while theta check should ensure that LLP SHOULD pass through, sometimes it still might not
        #       due on numerical precision etc - all these sanity checks are built in at every stage)
        if (llp_theta > theta_min) and (llp_theta < theta_max):
            # if it does, randomly rotate the phi of the LLP 4-vector
            four_p_rot = deal_with_phi(four_p, phi_min, phi_max)
            # use new 4-vector, lifetime of LLP, and the detector to generate
            # weight, decay position, boost of the LLP
            pack = get_weight(four_p_rot, m, ctau, detector_benchmark) # returns weight, decay_position, boost           
            
            # if get_weight does not return None i.e. LLP enter and exits detector (and there is no numerical precision issue etc.)
            if pack is not None:
                # for perfect decay, don't care about detection, just add (probabilty of decay * weight) to efficiency dictionary
                # this is why dont have any condition on decay such as reconstructed() etc.
                eff["perfect"] += pack[0] * w 

                ## determine if decay detected w.r.t. criterions or not
                
                # once again, clear the detector of previous events (if any)
                detector_benchmark.clear_detector()
                
                # randomly chose a decay based on branching ratio using possible_brs dictionary
                chosen_decay = random.choices(list(possible_brs.keys()), list(possible_brs.values()), k=1)[0]

                ## now get llp from the LLP gun depending on chosen decay
                ## NOTE: the last argument (i.e. decay_product) to lg.get_llp(type of decay, mass of LLP, decay position, boost, decay_product) 
                ## changes based on type of decay needed as follows:
                
                ## two-body decay to particle/anti-particle pair: 
                ## the last argument is list of PIDs of decay products e.g. [11,-11]
                
                ## three-body decay to particle/anti-particle/(anti-)neutrino:
                ## the last argument is list of PIDs of decay products e.g. [11,-11,12]
                
                ## hadronic decay:
                ## either empty for RHN or the type of hadronic decay (e.g. "ss") for SM+S
                
                ## change paths in llp_gun_new.py to those on the local machine (see llp_gun_new.py)
                
                if chosen_decay == "e":
                    # should be [11,-11] below but we "cut corners" and just feed in [11,11] as only the fact
                    # that a particle is charged or not matters, not the actual value of the charge
                    llp = lg.get_llp("leptonic2body", m, pack[1], pack[2], [11,11])

                elif chosen_decay == "mu": 
                    llp = lg.get_llp("leptonic2body", m, pack[1], pack[2], [13,13])

                elif chosen_decay == "pi": # pi means pi+-
                    if 2*m_pi0 <= m < 2*m_pi:
                        llp = None
                    else:
                        x = random.choices(["+","0"], [2/3,1/3], k=1)[0]
                        if x == "+": 
                            llp = lg.get_llp("leptonic2body", m, pack[1], pack[2], [211,211])

                        else:
                            llp = None

                elif chosen_decay == "hadrons0.7to0.98gev": 
                    llp = lg.get_llp("hadronic_SMS", m, pack[1], pack[2],"hadrons0.7to0.98gev")
                    # hadronic_SMS because we are doing the SM+S case
                    # change to hadronic_RHN_Ui or hadronic_HXX depending on the LLP case
                    # please look at the llp_gun_new files for function inputs depending on the LLP case

                elif chosen_decay == "hadrons1to2gev": 
                    llp = lg.get_llp("hadronic_SMS", m, pack[1], pack[2],"hadrons1to2gev")

                elif chosen_decay == "g": 
                    llp = lg.get_llp("hadronic_SMS", m, pack[1], pack[2],"gg")

                elif chosen_decay == "c":
                    llp = lg.get_llp("hadronic_SMS", m, pack[1], pack[2],"cc")

                elif chosen_decay == "s":
                    llp = lg.get_llp("hadronic_SMS", m, pack[1], pack[2],"ss")

                else: # tau-tau
                    llp = lg.get_llp("hadronic_SMS", m, pack[1], pack[2],"tautau")

                # make sure LLP is not None i.e. not an invisible decay
                if llp is not None:
                    detector_benchmark.new_vertex_event(llp)
                    
                    # show/save event display (if needed) using the following (uncomment to see)
                    # setting argument show = True displays and saves
                    # show = False saves but does not display
#                     filename = "/Users/jai/Desktop/figs/CDR_benchmark_vertex_LLPgun_" + str(k)
#                     detector_benchmark.detector_display(filename, show=True)
                    
                    # this section below is for when reconstruction matters (i.e. drop the perfect assumption)
                    for typ in eff: # typ is reconstruction criteria which form the keys of eff dictionary
                        if typ != "perfect":
                            # detector_benchmark.vertex_reconstructed(recon_criteria=typ) is 1 
                            # if LLP vertex reconstructed using given critera, 0 otherwise 
                            
                            # detector_benchmark.event_pass_trigger(trigger_criteria) is 1 
                            # if LLP vertex passes trigger criteria, 0 otherwise 
                            # NOTE: we use detector_benchmark.event_pass_trigger() without any argument as there is only one criteria
                            # and the simulation automatically uses that as default
                            
                            # if recon only is needed, delete/comment out the pass_trigger function
                            eff[typ] += pack[0] * w * detector_benchmark.vertex_reconstructed(recon_criteria=typ) * detector_benchmark.event_pass_trigger()


    ##################################
    # using efficiencies to get N_obs i.e. number of LLPs observed 
    
    # N_obs(criteria) = xsec * lumi * eff(criteria) * (phi_min - phi_max)/(2pi)
    # last factor needed to correct for random phi rotation about beam-axis
    
    eps_phi = (phi_max-phi_min)/(2*np.pi)
    
    N_obs = {"perfect":0,"DVmedium2":0,'DVmedium3':0,"DVmedium2loose1":0,"DVtight2":0,
          'DVsupertight2':0,'DVtight1medium1':0,'DVtight1loose1':0,'DVtight3':0,
          'DVtight2medium1':0,'DVtight1medium2':0,'DVtight1loose2':0}

    for typ in N_obs:
        N_obs[typ] = eff[typ] * eps_phi 
        
    # let's see if the sim worked
    print("mass:", m)
    print("coupling:", theta, "\n")
    for typ in N_obs:
        print("N_obs for", typ, ":", N_obs[typ], "\n")

## Converting hadronic decay 4-vectors into Fastim format

While we have provided links to repositories containing the hadronic decay products required for various LLP analyses, the FastSim takes them as input in a specific format (for e.g. look at file "bb_15.txt" or "SMS_LLPweight4vectorBmesonlist_mS_1.0.csv"). Here we demo the script to convert the given files into such a format.

In [1]:
from script import *

# we take in file bb_15 but in the geant (given) format and convert it into the format used in the section "VERTEX FROM FILE" 

# first argument is where to save output, second argument is where to read in the file to be converted
# change paths as needed
write_hadrons_for_fastsim("/Users/jai/Desktop/temp.txt", "/Users/jai/Desktop/final_data_aug_5/geant_decays/H_hadronic_decays_geant/bb_15.txt")